In [5]:
import types

In [9]:
isinstance(1, types.FunctionType)

False

In [13]:
import types

class Node(object):
    '''Node of computational graph.

    This object represents a node of a computation graph. A node has one
    function and several arguments of that function. At the time when the node
    is generated, the function is not applied, and it is calculated as
    necessary. A node can be an argument of another node.

    Args:
        func (function): The function applied to the argument.
        args (Node, list or tuple): Arguments of the function.

    Attributes:
        data: Computation result of this Node.
    '''
    def __init__(self, func, args):
        if not isinstance(func, types.FunctionType):
            raise TypeError('func mast be a function.')
        self.func = func
        self._data = None
        _args = []
        if not isinstance(args, (tuple, list)):
            args = [args]
        for arg in args:
            if not isinstance(arg, Node):
                raise TypeError('args must be Node or list(tuple) of Nodes.')
            _args.append(arg)
        self.args = tuple(_args)

    def apply_func(self):
        expanded_args = [arg.data for arg in self.args]
        return self.func(*expanded_args)
    
    @property
    def data(self):
        if self._data is None:
            self._data = self.apply_func()
        return self._data


class Variable(Node):
    '''Start node of computational graph.
    
    Computation graph starts from this object.
    
    Args:
        data: value of the Variable.
    '''
    def __init__(self, data):
        self.func = None
        self.args = ()
        self._data = data

    def apply_func(self):
        raise NotImplementedError()
    
    @property
    def data(self):
        return self._data

    @data.setter
    def data(self, d):
        self._data = d


In [2]:
def add(a, b):
    return a + b

def mul(a, b):
    return a * b

In [5]:
x = Variable(5)

h0 = Node(mul, [x, 5])
h1 = Node(mul, [Variable(2), x])
h2 = Node(add, [h0, h1])
y  = Node(add, [h2, Variable(1)])

In [14]:
print(h1._data)
print(y.data)
print(h1._data)

None
36
10


計算グラフの構築と計算の実行が分離されており、なおかつそのことを使用者に意識させない計算グラフフレームワーク。

Node は関数の適用を表現する。つまり関数とその引数の組を保持しており、必要に応じて実際に計算し値を返す。引数もまた Node である。ただし計算のたびに関数と引数の組を与えて新たな Node を作るというルールは、実運用上少々面倒である。したがって Node の生成をサポートする関数群を定義しておく。これらの関数は F(node) という形で新たな Node インスタンスを生成する。

In [12]:
h0.args[1].args

()